In [ ]:
import pandas as pd

# Pull the known results
df_fixtures = pd.read_csv("https://fixturedownload.com/download/fifa-world-cup-2022-UTC.csv")
df_results = df_fixtures[df_fixtures["Result"].notna()]

# Split Result into home goals and away goals
df_results["Home Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[0]))
df_results["Away Goals"] = df_results["Result"].apply(lambda x: int(x.split("-")[1]))

# Get the total goals for each team
df_goals = pd.concat(
    [
        df_results[[f"{x} Team", f"{x} Goals"]].rename(columns = lambda x: x.split(" ")[1]) 
        for x in ["Home", "Away"]
    ]
)
df_goals = df_goals.groupby("Team")[["Goals"]].sum().reset_index()

# Read the players picks
df_players_picks = pd.read_csv("df_players_picks.csv")

# Clean the players picks
df_players_picks = df_players_picks.set_index("name")
df_players_picks["picks"] = df_players_picks["picks"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))


# Convert the picks into seperate columns
df_players_goals = pd.DataFrame(df_players_picks["picks"].str.split(", ").tolist(), index=df_players_picks.index)

# Convert to long
df_players_goals = df_players_goals.stack().to_frame("Team")
df_players_goals.index = df_players_goals.index.droplevel(1)

# Join the goals and aggregate to each player
df_players_goals = pd.merge(df_players_goals.reset_index(), df_goals, on="Team", how="left").fillna(0)
df_players_goals = df_players_goals.groupby("name")[["Goals"]].sum().astype(int)

# Join the total goals the the original dataframe
df_players_picks_with_goals = df_players_picks.join(df_players_goals)

df_players_picks_with_goals